In [ ]:
from openai import OpenAI

client = OpenAI()

def obtener_analisis_completo(mensaje, empresa, tamaño_empresa):
    prompt = f"""
Eres un experto en ventas y análisis de clientes. Analiza el siguiente lead y responde con:

1. Lead Score (1 a 5): nivel de intención de compra (1 = interés muy bajo, 5 = interés muy alto).
2. Justificación: breve análisis del mensaje.
3. Categoría: tipo de servicio o necesidad detectada.
4. Recomendación: acción sugerida (ej. contactar ya, enviar info, ignorar, etc.)

Lead:
- Empresa: {empresa}
- Tamaño: {tamaño_empresa}
- Mensaje: "{mensaje}"

Responde en formato JSON:
{{
  "score": número del 1 al 5,
  "justificación": "...",



  
  "categoría": "...",
  "recomendación": "..."
}}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        content = response.choices[0].message.content.strip()
        return eval(content)  # O usar json.loads() si asegurás el formato
    except Exception as e:
        print(f"❌ Error con lead: {mensaje}\n{e}")
        return {
            "score": None,
            "justificación": "Error al analizar.",
            "categoría": "Desconocido",
            "recomendación": "Revisar manualmente"
        }


: 

In [ ]:
# Detectar necesidad a través de la IA
def detectar_necesidad_con_ia(mensaje):
    prompt = f"""
Analiza el siguiente mensaje de un lead y responde con una sola palabra o frase que describa el tipo de necesidad o servicio que busca.
Puede ser: "E-commerce", "Sitio Web", "Redes Sociales", "SEO", "Publicidad", "Diseño Gráfico", "Consultoría", etc.

Mensaje:
"{mensaje}"

Responde solo con el nombre de la categoría detectada.
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        st.warning(f"⚠️ Error detectando necesidad: '{mensaje[:40]}...'\n{e}")
        return "Desconocido"


In [ ]:
from openai import OpenAI

client = OpenAI()

def obtener_score(mensaje, empresa, tamaño_empresa):
    prompt = f"""
Eres un asesor experto en marketing digital. Evalúa la calidad del siguiente lead que solicita servicios relacionados con páginas web, e-commerce o redes sociales.

Del 1 al 5, indica la probabilidad de que este lead quiera contratar un servicio (1 = interés bajo, 5 = interés alto).


Lead:
- Empresa: {empresa}
- Tamaño: {tamaño_empresa}
- Mensaje: "{mensaje}"

Solo responde con un número del 1 al 5, sin explicaciones.
"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return int(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"❌ Error con lead: {mensaje}\n{e}")
        return None


In [ ]:
df = pd.read_csv("MicroCurso_respuestas.csv")

# Tomar solo las columnas relevantes del nuevo CSV
df_filtrado = pd.DataFrame({
    "nombre": df["Nombre.Email"].str.split(",").str[0],
    "email": df["Nombre.Email"].str.split(",").str[1],
    "empresa": "Sin datos",
    "tamaño_empresa": "pequeña",
    "mensaje": df[df.columns[-1]]  # última columna
})

df_filtrado["lead_score"] = df_filtrado.apply(
    lambda row: obtener_score(row["mensaje"], row["empresa"], row["tamaño_empresa"]),
    axis=1
)

df_filtrado.to_csv("leads_con_scores.csv", index=False)
df_filtrado


In [ ]:
def categorizar(score):
    if score >= 4:
        return "🟢 Caliente"
    elif score == 3:
        return "🟡 Tibio"
    elif score <= 2:
        return "🔴 Frío"
    else:
        return "❓"

df["categoría"] = df["lead_score"].apply(categorizar)
df.to_csv("leads_con_scores.csv", index=False)
df


In [ ]:
df.to_excel("leads_con_scores.xlsx", index=False)
